# GAMA-15 master catalogue
## Preparation of DECam Legacy Survey data

This catalogue comes from `dmu0_DECaLS`.

In the catalogue, we keep:

- The `object_id` as unique object identifier;
- The position;
- The u, g, r, i, z, Y aperture magnitude (2”);
- The u, g, r, i, z, Y kron fluxes and magnitudes.

We check for all ugrizY then only take bands for which there are measurements

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
plt.style.use('ggplot')

from collections import OrderedDict
import os

from astropy import units as u
from astropy import visualization as vis
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, nb_plot_mag_ap_evol, \
    nb_plot_mag_vs_apcor, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux, aperture_correction, flux_to_mag

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "decals_ra"
DEC_COL = "decals_dec"

In [ ]:
# Pritine LS catalogue
orig_decals = Table.read("../../dmu0/dmu0_DECaLS/data/DECaLS_GAMA-15.fits")

## I - Aperture correction

To compute aperture correction we need to dertermine two parametres: the target aperture and the range of magnitudes for the stars that will be used to compute the correction.

**Target aperture**: To determine the target aperture, we simulate a curve of growth using the provided apertures and draw two figures:

- The evolution of the magnitudes of the objects by plotting on the same plot aperture number vs the mean magnitude.
- The mean gain (loss when negative) of magnitude is each aperture compared to the previous (except for the first of course).

As target aperture, we should use the smallest (i.e. less noisy) aperture for which most of the flux is captures.

**Magnitude range**: To know what limits in aperture to use when doing the aperture correction, we plot for each magnitude bin the correction that is computed and its RMS. We should then use the wide limits (to use more stars) where the correction is stable and with few dispersion.

In [ ]:
bands = ["u", "g", "r", "i", "z", "y"]
band_index = {"u":0,"g":1, "r":2, "i":3, "z":4, "y":5}
apertures      = [0,      1,   2,   3,    4,   5,   6,   7] 
aperture_sizes = [0.5, 0.75, 1.0, 1.5,  2.0, 3.5, 5.0, 7.0] #arcsec aperture sizes

flux = {}
flux_errors ={}
magnitudes = {}
flux_errors ={}
magnitude_errors = {}
stellarities = {}

flux_to_mag_vect = np.vectorize(flux_to_mag)

for band in bands:
    flux[band] = np.transpose(np.array(orig_decals["decam_apflux"][:,band_index[band]])) #np.transpose(np.array( orig_decals["decam_apflux"], dtype=np.float )) 
    flux_errors[band] = np.transpose(np.array(orig_decals["decam_apflux_ivar"][:,band_index[band]])) #np.transpose(np.array( orig_legacy["apflux_ivar_{}".format(band)], dtype=np.float  ))
    
    magnitudes[band], magnitude_errors[band] = flux_to_mag_vect(flux[band] * 3.631e-6 ,flux_errors[band] * 3.631e-6)
    
    stellarities[band] = np.full(len(orig_decals),0., dtype='float32')
    stellarities[band][np.array( orig_decals["type"]) == "PSF " ] = 1.
    stellarities[band][np.array( orig_decals["type"]) == "PSF" ] = 1.
    
    # Some sources have an infinite magnitude
    mask = np.isinf(magnitudes[band])
    magnitudes[band][mask] = np.nan
    magnitude_errors[band][mask] = np.nan
    

    
    
mag_corr = {}

### 1.a u band

In [ ]:
nb_plot_mag_ap_evol(magnitudes['u'], stellarities['u'], labels=apertures)

u band is all nan

In [ ]:
nb_plot_mag_vs_apcor(magnitudes['u'][4], 
                     magnitudes['u'][5], 
                     stellarities['u'])

In [ ]:
# Aperture correction
mag_corr['u'] = np.nan
#mag_corr['u'], num, std = aperture_correction(
#    magnitudes['u'][4], magnitudes['u'][5], 
#    stellarities['u'],
#    mag_min=16.0, mag_max=19.0)
#print("Aperture correction for g band:")
#print("Correction: {}".format(mag_corr['g']))
#print("Number of source used: {}".format(num))
#print("RMS: {}".format(std))

### I.a - g band

In [ ]:
nb_plot_mag_ap_evol(magnitudes['g'], stellarities['g'], labels=apertures)

We will use aperture 5 as target.

In [ ]:
nb_plot_mag_vs_apcor(magnitudes['g'][4], 
                     magnitudes['g'][5], 
                     stellarities['g'])

We will use magnitudes between 16.0 and 19.0

In [ ]:
# Aperture correction
#mag_corr['g'] = np.nan
mag_corr['g'], num, std = aperture_correction(
    magnitudes['g'][4], magnitudes['g'][5], 
    stellarities['g'],
    mag_min=16.0, mag_max=19.0)
print("Aperture correction for g band:")
print("Correction: {}".format(mag_corr['g']))
print("Number of source used: {}".format(num)) 
print("RMS: {}".format(std))


### I.b - r band

In [ ]:
nb_plot_mag_ap_evol(magnitudes['r'], stellarities['r'], labels=apertures)

We will use aperture 5 as target.

In [ ]:
nb_plot_mag_vs_apcor(magnitudes['r'][4], 
                     magnitudes['r'][5], 
                     stellarities['r'])

We use magnitudes between 16.0 and 18.0.

In [ ]:
# Aperture correction
#mag_corr['r'] = np.nan
mag_corr['r'], num, std = aperture_correction(
    magnitudes['r'][4], magnitudes['r'][5], 
    stellarities['r'],
    mag_min=16.0, mag_max=18.0)
print("Aperture correction for r band:")
print("Correction: {}".format(mag_corr['r']))
print("Number of source used: {}".format(num))
print("RMS: {}".format(std))

### I.d - i band

In [ ]:
nb_plot_mag_ap_evol(magnitudes['i'], stellarities['i'], labels=apertures)

In [ ]:
nb_plot_mag_vs_apcor(magnitudes['i'][4], 
                     magnitudes['i'][4], 
                     stellarities['i'])

In [ ]:
# Aperture correction
mag_corr['i'] = np.nan
#mag_corr['i'], num, std = aperture_correction(
#    magnitudes['i'][4], magnitudes['i'][5], 
#    stellarities['i'],
#    mag_min=16.0, mag_max=17.5)
#print("Aperture correction for i band:")
#print("Correction: {}".format(mag_corr['i']))
#print("Number of source used: {}".format(num))
#print("RMS: {}".format(std))

### I.e - z band

In [ ]:
nb_plot_mag_ap_evol(magnitudes['z'], stellarities['z'], labels=apertures)

We will use aperture 4 as target.

In [ ]:
nb_plot_mag_vs_apcor(magnitudes['z'][4], 
                     magnitudes['z'][4], 
                     stellarities['z'])

We use magnitudes between 16.0 and 17.5.

In [ ]:
# Aperture correction
#mag_corr['z'] = np.nan
mag_corr['z'], num, std = aperture_correction(
    magnitudes['z'][4], magnitudes['z'][4], 
    stellarities['z'],
    mag_min=16.0, mag_max=17.5)
print("Aperture correction for z band:")
print("Correction: {}".format(mag_corr['z']))
print("Number of source used: {}".format(num))
print("RMS: {}".format(std))

### I.f - Y band

In [ ]:
nb_plot_mag_ap_evol(magnitudes['y'], stellarities['y'], labels=apertures)

In [ ]:
nb_plot_mag_vs_apcor(magnitudes['y'][4], 
                     magnitudes['y'][4], 
                     stellarities['y'])

In [ ]:
# Aperture correction
mag_corr['y'] = np.nan
#mag_corr['y'], num, std = aperture_correction(
#    magnitudes['y'][4], magnitudes['y'][5], 
#    stellarities['y'],
#    mag_min=16.0, mag_max=17.5)
#print("Aperture correction for y band:")
#print("Correction: {}".format(mag_corr['y']))
#print("Number of source used: {}".format(num))
#print("RMS: {}".format(std))


## II - Stellarity

Legacy Survey does not provide a 0 to 1 stellarity so we replace items flagged as PSF accpording to the following table:

\begin{equation*}
P(star) = \frac{ \prod_{i} P(star)_i }{ \prod_{i} P(star)_i + \prod_{i} P(galaxy)_i }
\end{equation*}

where $i$ is the band, and with using the same probabilities as UKDISS:

| HSC flag | UKIDSS flag | Meaning         | P(star) | P(galaxy) | P(noise) | P(saturated) |
|:--------:|:-----------:|:----------------|--------:|----------:|---------:|-------------:|
|          |      -9     | Saturated       |     0.0 |       0.0 |      5.0 |         95.0 |
|          |      -3     | Probable galaxy |    25.0 |      70.0 |      5.0 |          0.0 |
|          |      -2     | Probable star   |    70.0 |      25.0 |      5.0 |          0.0 |
|     0    |      -1     | Star            |    90.0 |       5.0 |      5.0 |          0.0 |
|          |      0      | Noise           |     5.0 |       5.0 |     90.0 |          0.0 |
|     1    |      +1     | Galaxy          |     5.0 |      90.0 |      5.0 |          0.0 |

In [ ]:
stellarities['g'][np.isclose(stellarities['g'], 1.)] = 0.9
stellarities['g'][np.isclose(stellarities['g'], 0.)] = 0.05

## II - Column selection

In [ ]:
imported_columns = OrderedDict({
        "objid": "decals_id",
        "brickid": "brickid",
        "ra": "decals_ra",
        "dec": "decals_dec",
        "decam_flux": "decam_flux_TEMP",
        "decam_flux_ivar": "decam_flux_ivar_TEMP",
        "decam_apflux": "decam_apflux_TEMP",
        "decam_apflux_ivar": "decam_apflux_ivar_TEMP",
       
    })


catalogue = Table.read("../../dmu0/dmu0_DECaLS/data/DECaLS_GAMA-15.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]
    
catalogue["decals_id"] = 100000*catalogue["brickid"].astype(np.int64) + catalogue["decals_id"].astype(np.int64)
catalogue.remove_columns("brickid")

epoch = 2017

#catalogue.add_column(Column(catalogue["decam_flux_TEMP"][:,0], name="f_decam_u"))
catalogue.add_column(Column(catalogue["decam_flux_TEMP"][:,1], name="f_decam_g"))
catalogue.add_column(Column(catalogue["decam_flux_TEMP"][:,2], name="f_decam_r"))
#catalogue.add_column(Column(catalogue["decam_flux_TEMP"][:,3], name="f_decam_i"))
catalogue.add_column(Column(catalogue["decam_flux_TEMP"][:,4], name="f_decam_z"))
#catalogue.add_column(Column(catalogue["decam_flux_TEMP"][:,5], name="f_decam_y"))

#catalogue.add_column(Column(catalogue["decam_flux_ivar_TEMP"][:,0], name="ferr_decam_u"))
catalogue.add_column(Column(catalogue["decam_flux_ivar_TEMP"][:,1], name="ferr_decam_g"))
catalogue.add_column(Column(catalogue["decam_flux_ivar_TEMP"][:,2], name="ferr_decam_r"))
#catalogue.add_column(Column(catalogue["decam_flux_ivar_TEMP"][:,3], name="ferr_decam_i"))
catalogue.add_column(Column(catalogue["decam_flux_ivar_TEMP"][:,4], name="ferr_decam_z"))
#catalogue.add_column(Column(catalogue["decam_flux_ivar_TEMP"][:,5], name="ferr_decam_y"))

#For the aperture fluxes, there are 8 (0-7), we take 4 (2.0")
#DECam aperture fluxes on the co-added images in apertures of radius [0.5,0.75,1.0,1.5,2.0,3.5,5.0,7.0] arcsec in ugrizY
#catalogue.add_column(Column(catalogue["decam_apflux_TEMP"][:,0], name="f_ap_decam_u")[:,4])
catalogue.add_column(Column(catalogue["decam_apflux_TEMP"][:,1], name="f_ap_decam_g")[:,4])
catalogue.add_column(Column(catalogue["decam_apflux_TEMP"][:,2], name="f_ap_decam_r")[:,4])
#catalogue.add_column(Column(catalogue["decam_apflux_TEMP"][:,3], name="f_ap_decam_i")[:,4])
catalogue.add_column(Column(catalogue["decam_apflux_TEMP"][:,4], name="f_ap_decam_z")[:,4])
#catalogue.add_column(Column(catalogue["decam_apflux_TEMP"][:,5], name="f_ap_decam_y")[:,4])

#catalogue.add_column(Column(catalogue["decam_apflux_ivar_TEMP"][:,0], name="ferr_ap_decam_u")[:,4])
catalogue.add_column(Column(catalogue["decam_apflux_ivar_TEMP"][:,1], name="ferr_ap_decam_g")[:,4])
catalogue.add_column(Column(catalogue["decam_apflux_ivar_TEMP"][:,2], name="ferr_ap_decam_r")[:,4])
#catalogue.add_column(Column(catalogue["decam_apflux_ivar_TEMP"][:,3], name="ferr_ap_decam_i")[:,4])
catalogue.add_column(Column(catalogue["decam_apflux_ivar_TEMP"][:,4], name="ferr_ap_decam_z")[:,4])
#catalogue.add_column(Column(catalogue["decam_apflux_ivar_TEMP"][:,5], name="ferr_ap_decam_y")[:,4])

catalogue.remove_columns(["decam_flux_TEMP", 
                          "decam_flux_ivar_TEMP", 
                          "decam_apflux_TEMP", 
                          "decam_apflux_ivar_TEMP"])

# Clean table metadata
catalogue.meta = None

In [ ]:
flux_to_mag_vect = np.vectorize(flux_to_mag)


# Adding flux and band-flag columns
for col in catalogue.colnames:
    catalogue[col].unit = None
    if col.startswith('f_'):
        
        #Replace 0 flux with NaN and 
        catalogue[col][catalogue[col] == 0.0] = np.nan
        
        #Replace 1/sigma^2 with sigma 
        errcol = "ferr{}".format(col[1:])
        catalogue[errcol][catalogue[errcol] == 0.0] = np.nan
        catalogue[errcol] = np.sqrt(1/np.array(catalogue[errcol]))
        #catalogue[errcol][catalogue[errcol] == None] = np.nan
        
        #Replace nanomaggies with uJy
        #a nanomaggy is approximately 3.631×10-6 Jy - http://www.sdss3.org/dr8/algorithms/magnitudes.php#nmgy
        catalogue[col] = catalogue[col]  * 3.631        
        catalogue[errcol] = catalogue[errcol]  * 3.631
        
        #Compute magnitudes and errors in magnitudes. This function expects Jy so must multiply uJy by 1.e-6
        mag, error = flux_to_mag(np.array(catalogue[col])* 1.e-6, np.array(catalogue[errcol])* 1.e-6)
        
        if 'ap' in col:
            mag += mag_corr[col[-1]]
            catalogue[col],catalogue[errcol] = mag_to_flux(mag,error)
        
        catalogue.add_column(Column(mag, name="m{}".format(col[1:])))
        catalogue.add_column(Column(error, name="m{}".format(errcol[1:])))
        
        # Band-flag column
        if 'ap' not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        
#remove units from table
for col in catalogue.colnames:
    catalogue[col].unit = None
    
catalogue.add_column(Column(data=stellarities['g'], name="decals_stellarity")) #Stelarites computed earlier

In [ ]:
catalogue[:10].show_in_notebook()

## III - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = [#'merr_ap_decam_u',
             'merr_ap_decam_g',
             'merr_ap_decam_r',
             #'merr_ap_decam_i',
             'merr_ap_decam_z',
             #'merr_ap_decam_y'
]
FLAG_NAME = 'decals_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(
    catalogue, RA_COL, DEC_COL, 
    sort_col= SORT_COLS,
    flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_GAMA-15.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] = catalogue[RA_COL] +  delta_ra.to(u.deg)
catalogue[DEC_COL] = catalogue[DEC_COL] +  delta_dec.to(u.deg)
catalogue[RA_COL].unit = u.deg
catalogue[DEC_COL].unit = u.deg

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec, near_ra0=True)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "decals_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

# V - Saving to disk

In [ ]:
catalogue.write("{}/DECaLS.fits".format(OUT_DIR), overwrite=True)